In [1]:
import pandas as pd
import draft.data_cleaning as dc

In [2]:
person_0 = pd.read_json(r'datasets/december/liza.json', orient='records', lines=True) # liza
person_1 = pd.read_json(r'datasets/december/sleep_data_Adham.json', lines=True) # adham
person_2 = pd.read_json(r'datasets/december/sleep_data_Miriam.json', lines=True) # miriam
person_3 = pd.read_json(r'datasets/december/sleep_data_Syahid.json', lines=True) # syahid
person_4 = pd.read_json(r'datasets/december/sleep_data_Florian.json', lines=True) # florian
person_5 = pd.read_json(r'datasets/december/sleep_data_Shado.json', lines=True) # shado
person_6 = pd.read_json(r'datasets/december/sleep_data_Alina.json', lines=True) # alina

# labels dataframe from excel
labels_df = pd.read_excel(r'datasets\sleep_data.xlsx', sheet_name=None) # dict of all label

In [3]:
people_df = [person_0, person_1, person_2, person_3, person_4, person_5, person_6]

for num, df in enumerate(people_df):
    df.insert(0, "temp_id", num)

[embeddings + static data] + label --> RF (Leute mit Label)

In [4]:
def get_static_data(df):
    # Extract temp_id
    temp_id = df['temp_id']

    # static data from dailySleepDTO json
    sleepDTO_df = pd.json_normalize(df['dailySleepDTO'])

    cols_to_keep = ['id', 'calendarDate', 'sleepTimeSeconds', 'sleepStartTimestampLocal',
       'sleepEndTimestampLocal', 'deepSleepSeconds', 'lightSleepSeconds',
       'remSleepSeconds', 'awakeSleepSeconds', 'averageRespirationValue',
       'lowestRespirationValue', 'highestRespirationValue', 'awakeCount',
       'avgSleepStress', 'ageGroup', 'sleepVersion',
       'sleepScores.totalDuration.qualifierKey',
       'sleepScores.stress.qualifierKey',
       'sleepScores.awakeCount.qualifierKey', 'sleepScores.overall.value',
       'sleepScores.remPercentage.value',
       'sleepScores.remPercentage.idealStartInSeconds',
       'sleepScores.remPercentage.idealEndInSeconds',
       'sleepScores.restlessness.qualifierKey',
       'sleepScores.lightPercentage.value',
       'sleepScores.lightPercentage.idealStartInSeconds',
       'sleepScores.lightPercentage.idealEndInSeconds',
       'sleepScores.deepPercentage.value',
       'sleepScores.deepPercentage.idealStartInSeconds',
       'sleepScores.deepPercentage.idealEndInSeconds']
    
    sleepDTO_df = sleepDTO_df[cols_to_keep]

    # Add temp_id to the sleepDTO_df
    sleepDTO_df['temp_id'] = temp_id 

    # change the time format
    sleepDTO_df['sleepStartTimestampLocal'] = pd.to_datetime(sleepDTO_df['sleepStartTimestampLocal'], unit='ms')
    sleepDTO_df['sleepEndTimestampLocal'] = pd.to_datetime(sleepDTO_df['sleepEndTimestampLocal'], unit='ms')

    return sleepDTO_df

In [5]:
def combine_static_data(df):
    # Process the sleepDTO data
    sleepDTO = get_static_data(df)

    # Extract the required static values from the original DataFrame
    cols = ['temp_id', 'remSleepData', 'restlessMomentsCount', 'avgOvernightHrv', 'restingHeartRate']
    static_values = df[cols]

    # Set 'temp_id' as the index for both DataFrames
    static_values.set_index('temp_id', inplace=True)
    sleepDTO.set_index('temp_id', inplace=True)

    # Concatenate static_values and sleepDTO horizontally
    combined_data = pd.concat([sleepDTO, static_values], axis=1)

    # Reset the index if you want 'temp_id' back as a column
    combined_data.reset_index(inplace=True)

    return combined_data

In [6]:
def process_dataframe(dataframe):
    for col in dataframe.columns:
        if col.endswith("qualifierKey"):
            new_name = col.replace("qualifierKey", "value")
            dataframe[new_name] = dataframe[col].apply(dc.convert_num)
            dataframe.drop(col, axis=1, inplace=True)
    
    return dataframe

sleepDTO --> drop irrelavnt columns --> change qualifier key to integer
static values 
labels

In [13]:
labels_df['Shado']

,Datum,ausgeschlafen,motivation,konzentriert,Wach,Test_zeit,Test_anzahl,prozent_zeit_rang,prozent_anzahl
0,2023-12-08,3,5,5,3,52.00,95,0,72
1,2023-12-09,4,5,5,2,51.67,100,0,90
2,2023-12-10,4,4,2,2,51.67,100,0,90
3,2023-12-11,2,4,5,2,46.33,100,0,90
4,2023-12-12,4,5,5,2,38.00,100,0,90
5,2023-12-13,4,4,5,3,38.00,100,0,90
6,2023-12-14,3,4,4,2,40.33,100,0,90
7,2023-12-15,5,5,5,3,37.67,100,0,90
8,2023-12-16,5,5,5,2,38.00,99,0,87
9,2023-12-17,4,5,5,3,34.67,100,0,90


In [7]:
people_dict = {0: 'Liza', 1: 'Adham', 2: 'Miriam', 3: 'Syahid', 4: 'Florian', 5: 'Shado', 6: 'Alina'} # excel

persons_static_data = {
    0 : person_0,
    1 : person_1,
    2 : person_2,
    3 : person_3,
    4 : person_4,
    5 : person_5,
    6 : person_6
}

def process_all_people(persons_static_data): # we put persons_data as the argument
    processed_people = {}

    for person_id, person_df in persons_static_data.items():
        # Clean the data for each person
        cleaned_df = dc.delete_untracked_nights(person_df)

        # Combine the static data
        combined_df = combine_static_data(cleaned_df)

        # Process the DataFrame to change 'qualifierKey' to numerical value
        final_processed_df = process_dataframe(combined_df)

        # Load the embeddings DataFrame from a pickle file according to person id
        embeddings_file_name = f"datasets/embeddings/embeddings_{person_id}.pkl"
        embeddings_df = pd.read_pickle(embeddings_file_name)

        # combine the static and the embeddings
        combined_df_with_embeddings = pd.concat([final_processed_df, embeddings_df], axis=1)

        # Store the processed DataFrame in the dictionary with the same person_id
        processed_people[person_id] = combined_df_with_embeddings

    return processed_people

In [8]:
all_df = process_all_people(persons_static_data)

In [16]:
all_df[0] # data from liza

,temp_id,id,calendarDate,sleepTimeSeconds,sleepStartTimestampLocal,sleepEndTimestampLocal,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,...,embedding_1990,embedding_1991,embedding_1992,embedding_1993,embedding_1994,embedding_1995,embedding_1996,embedding_1997,embedding_1998,embedding_1999
0,0,1701986940000,2023-12-08,36780,2023-12-07 23:09:00,2023-12-08 09:34:00,3780,27000,6000,720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1702078860000,2023-12-09,29940,2023-12-09 00:41:00,2023-12-09 09:42:00,3240,24480,2220,2520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1702171920000,2023-12-10,25080,2023-12-10 02:32:00,2023-12-10 09:39:00,2040,17580,5460,540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1702254240000,2023-12-11,21420,2023-12-11 01:24:00,2023-12-11 07:22:00,3720,15000,2700,60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1702340700000,2023-12-12,31080,2023-12-12 01:25:00,2023-12-12 10:04:00,6540,17520,7020,60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,1702510140000,2023-12-14,23880,2023-12-14 00:29:00,2023-12-14 07:21:00,5280,15600,3000,840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,1702596000000,2023-12-15,32100,2023-12-15 00:20:00,2023-12-15 09:27:00,3540,23520,5040,720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0,1702680600000,2023-12-16,38040,2023-12-15 23:50:00,2023-12-16 10:56:00,4500,22380,11160,1920,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0,1702777020000,2023-12-17,25260,2023-12-17 02:37:00,2023-12-17 09:44:00,3780,18000,3480,360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0,1702858680000,2023-12-18,30176,2023-12-18 01:18:00,2023-12-18 09:59:56,3000,22740,4380,1140,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
labels_df[people_dict[0]] # is same as labels_df['Liza']

,Datum,ausgeschlafen,motivation,konzentriert,Wach,Test_zeit,Test_anzahl,prozent_zeit_rang,prozent_anzahl
0,2023-12-08,4,3,4,2,NaN,NaN,NaN,NaN
1,2023-12-09,5,5,4,4,80.33,71.0,32.0,6.0
2,2023-12-10,4,2,3,2,79.33,27.0,33.0,7.0
3,2023-12-11,4,4,5,3,100.00,87.0,61.0,28.0
4,2023-12-12,5,5,4,2,85.00,81.0,32.0,6.0
5,2023-12-13,5,4,5,4,80.00,79.0,26.0,5.0
6,2023-12-14,3,4,3,1,73.33,92.0,18.0,56.0
7,2023-12-15,5,3,4,1,86.00,93.0,33.0,62.0
8,2023-12-16,5,5,5,5,69.33,93.0,8.0,62.0
9,2023-12-17,5,5,4,5,75.00,97.0,15.0,81.0


In [9]:
def merge_with_labels(persons_data, labels_df, people_dict):
    merged_data = {}

    for person_id, person_df in persons_data.items():
        # Check if 'calendarDate' exists in person_df
        if 'calendarDate' not in person_df.columns:
            print(f"'calendarDate' column not found in person data for {person_id}")
            continue

        # Get the label DataFrame for the current person
        person_label_df = labels_df[people_dict[person_id]]

        # Ensure the date columns are in the same format
        person_df['calendarDate'] = pd.to_datetime(person_df['calendarDate'])
        person_label_df['Datum'] = pd.to_datetime(person_label_df['Datum'])

        # Merge the person's data with their labels on the date
        merged_df = pd.merge(person_df, person_label_df, left_on='calendarDate', right_on='Datum')

        # Store the merged DataFrame in the dictionary with the same person_id
        merged_data[person_id] = merged_df

    return merged_data

# Example usage
merged_people_data = merge_with_labels(all_df, labels_df, people_dict)

In [19]:
all_df[3]

,temp_id,id,calendarDate,sleepTimeSeconds,sleepStartTimestampLocal,sleepEndTimestampLocal,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,...,embedding_1990,embedding_1991,embedding_1992,embedding_1993,embedding_1994,embedding_1995,embedding_1996,embedding_1997,embedding_1998,embedding_1999
0,3,1698804840000,2023-11-01,36194,2023-11-01 03:14:00,2023-11-01 13:59:14,4800,24600,6780,2520,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,3,1698878280000,2023-11-02,27600,2023-11-01 23:38:00,2023-11-02 07:20:00,3120,21600,2880,120,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,3,1698975540000,2023-11-03,31249,2023-11-03 02:39:00,2023-11-03 11:32:49,1380,23820,6000,780,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
3,3,1699062480000,2023-11-04,39967,2023-11-04 02:48:00,2023-11-04 14:28:07,2700,24240,13020,2040,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
4,3,1699152240000,2023-11-05,26160,2023-11-05 03:44:00,2023-11-05 11:01:00,6660,18840,660,60,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
5,3,1699229700000,2023-11-06,21054,2023-11-06 01:15:00,2023-11-06 07:25:54,3780,17220,0,1200,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
6,3,1699404300000,2023-11-08,35400,2023-11-08 01:45:00,2023-11-08 11:47:00,2940,23460,9000,720,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
7,3,1699491240000,2023-11-09,21592,2023-11-09 01:54:00,2023-11-09 08:00:52,4920,16620,0,420,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
8,3,1699580100000,2023-11-10,30960,2023-11-10 02:35:00,2023-11-10 11:36:00,2220,26100,2640,1500,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
9,3,1699691220000,2023-11-11,12180,2023-11-11 09:27:00,2023-11-11 12:58:00,3600,6360,2220,480,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [20]:
labels_df["Syahid"]

,Datum,ausgeschlafen,motivation,konzentriert,Wach,Test_zeit,Test_anzahl,prozent_zeit_rang,prozent_anzahl
0,2023-12-08,4,2,3,3,110.33,99,76,87
1,2023-12-09,4,3,1,3,116.00,100,82,90
2,2023-12-10,2,2,4,4,107.00,100,71,90
3,2023-12-11,4,4,5,1,127.33,100,91,90
4,2023-12-12,2,3,4,2,86.33,100,34,90
5,2023-12-13,4,4,4,2,81.67,99,25,87
6,2023-12-14,1,4,1,1,76.67,100,17,90
7,2023-12-15,2,3,3,2,71.33,100,10,90
8,2023-12-16,3,4,4,3,246.33,100,100,90
9,2023-12-17,4,3,3,3,722.67,100,100,90


In [21]:
# person 0 --> 4 NaN entries --> test label
# person 4 --> 8 NaN entries --> 4 columns can subtitute as '3'. Remaining is test label.
# person 6 --> 2 NaN entriies --> can subtitute as '3' (wach, konzentriert)

merged_people_data[6].isna().any().value_counts()

False    2044
Name: count, dtype: int64

In [22]:
merged_people_data[4]

,temp_id,id,calendarDate,sleepTimeSeconds,sleepStartTimestampLocal,sleepEndTimestampLocal,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,...,embedding_1999,Datum,ausgeschlafen,motivation,konzentriert,Wach,Test_zeit,Test_anzahl,prozent_zeit_rang,prozent_anzahl
0,4,1701988440000,2023-12-08,22800,2023-12-07 23:34:00,2023-12-08 06:21:00,2640,16980,3180,1620,...,0.0,2023-12-08,3,3,4,2,NaN,NaN,NaN,NaN
1,4,1702067940000,2023-12-09,35460,2023-12-08 21:39:00,2023-12-09 09:08:00,4620,23580,7260,5880,...,0.0,2023-12-09,4,1,3,2,NaN,NaN,NaN,NaN
2,4,1702169940000,2023-12-10,26220,2023-12-10 01:59:00,2023-12-10 09:29:00,2820,18300,5100,780,...,0.0,2023-12-10,2,2,2,3,60.00,93.0,2.0,62.0
3,4,1702246860000,2023-12-11,24780,2023-12-10 23:21:00,2023-12-11 06:16:00,3420,14520,6840,120,...,0.0,2023-12-11,2,3,3,3,61.33,93.0,3.0,62.0
4,4,1702334760000,2023-12-12,23460,2023-12-11 23:46:00,2023-12-12 06:45:00,5040,14100,4320,1680,...,0.0,2023-12-12,3,3,4,2,60.00,92.0,2.0,56.0
5,4,1702421040000,2023-12-13,23760,2023-12-12 23:44:00,2023-12-13 06:30:00,2460,15360,5940,600,...,0.0,2023-12-13,2,2,4,3,63.67,97.0,4.0,81.0
6,4,1702506960000,2023-12-14,22740,2023-12-13 23:36:00,2023-12-14 06:32:00,3120,14040,5580,2220,...,0.0,2023-12-14,3,3,4,3,51.67,97.0,0.0,81.0
7,4,1702677060000,2023-12-16,37140,2023-12-15 22:51:00,2023-12-16 09:36:00,7680,17940,11520,1560,...,0.0,2023-12-16,4,3,4,2,51.67,93.0,0.0,62.0
8,4,1702772280000,2023-12-17,28920,2023-12-17 01:18:00,2023-12-17 09:40:00,3960,19620,5340,1200,...,0.0,2023-12-17,4,5,4,3,55.67,99.0,1.0,87.0
9,4,1702853220000,2023-12-18,23460,2023-12-17 23:47:00,2023-12-18 06:24:00,5940,14100,3420,360,...,0.0,2023-12-18,3,3,5,4,61.33,97.0,3.0,81.0


In [10]:
data = pd.DataFrame()
# combine all data of all people
for df in merged_people_data.values():
    #print(df.shape)
    data = pd.concat([data, df])

data.drop('Datum', axis='columns', inplace=True)
data

,temp_id,id,calendarDate,sleepTimeSeconds,sleepStartTimestampLocal,sleepEndTimestampLocal,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,...,embedding_1998,embedding_1999,ausgeschlafen,motivation,konzentriert,Wach,Test_zeit,Test_anzahl,prozent_zeit_rang,prozent_anzahl
0,0,1701986940000,2023-12-08,36780,2023-12-07 23:09:00,2023-12-08 09:34:00,3780,27000,6000,720,...,0.0,0.0,4,3,4,2,NaN,NaN,NaN,NaN
1,0,1702078860000,2023-12-09,29940,2023-12-09 00:41:00,2023-12-09 09:42:00,3240,24480,2220,2520,...,0.0,0.0,5,5,4,4,80.33,71.0,32.0,6.0
2,0,1702171920000,2023-12-10,25080,2023-12-10 02:32:00,2023-12-10 09:39:00,2040,17580,5460,540,...,0.0,0.0,4,2,3,2,79.33,27.0,33.0,7.0
3,0,1702254240000,2023-12-11,21420,2023-12-11 01:24:00,2023-12-11 07:22:00,3720,15000,2700,60,...,0.0,0.0,4,4,5,3,100.00,87.0,61.0,28.0
4,0,1702340700000,2023-12-12,31080,2023-12-12 01:25:00,2023-12-12 10:04:00,6540,17520,7020,60,...,0.0,0.0,5,5,4,2,85.00,81.0,32.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,1702856640000,2023-12-18,22620,2023-12-18 00:44:00,2023-12-18 07:11:00,2940,15720,3960,600,...,0.0,0.0,3,3,3,3,51.00,97.0,0.0,81.0
6,6,1702940220000,2023-12-19,25800,2023-12-18 23:57:00,2023-12-19 07:10:00,2640,15420,7740,180,...,0.0,0.0,3,3,4,3,47.70,93.0,0.0,62.0
7,6,1703028000000,2023-12-20,26520,2023-12-20 00:20:00,2023-12-20 07:59:00,3420,15120,7980,1020,...,0.0,0.0,4,4,3,3,44.33,88.0,0.0,33.0
8,6,1703113800000,2023-12-21,28320,2023-12-21 00:10:00,2023-12-21 08:07:00,1200,19860,7260,300,...,0.0,0.0,5,5,3,3,44.70,95.0,0.0,72.0


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76 entries, 0 to 9
Columns: 2043 entries, temp_id to prozent_anzahl
dtypes: datetime64[ns](3), float32(2000), float64(18), int64(21), object(1)
memory usage: 619.9+ KB


In [17]:
data.isna().any().value_counts()

False    2040
True        4
Name: count, dtype: int64

In [74]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

data_copy = data.copy().dropna()

# list of columns that need to be dropped
to_drop = ['temp_id', 'id', 'calendarDate', 'ageGroup',
            'ausgeschlafen', 'motivation', 'Wach', 'konzentriert',
            'Test_zeit', 'Test_anzahl','prozent_zeit_rang', 'prozent_anzahl'] 

# train with the suitable dataset
features = data_copy.drop(to_drop, axis=1)  # Drop the target variable to isolate the features
target = data_copy['prozent_zeit_rang'].to_numpy()  # Isolate the target variable

# Convert Timestamp to Unix format
features['sleepStartTimestampLocal'] = features['sleepStartTimestampLocal'].astype('int64') // 10**9
 
features['sleepEndTimestampLocal'] = features['sleepEndTimestampLocal'].astype('int64') // 10**9
 
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# normalize the features values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
 
# Initialize the Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42)

parameters = {"n_estimators" : [20, 40, 60, 80],
              "max_depth": [5, 8, 10, 12, 15],
              "min_samples_leaf": [2, 4, 6, 8, 10],
              "n_estimators":[10, 50, 100]}
 
random_search = RandomizedSearchCV(rf_regressor, parameters, n_iter=20, verbose=2)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ..max_depth=10, min_samples_leaf=2, n_estimators=50; total time=   2.1s
[CV] END ..max_depth=10, min_samples_leaf=2, n_estimators=50; total time=   1.9s
[CV] END ..max_depth=10, min_samples_leaf=2, n_estimators=50; total time=   2.1s
[CV] END ..max_depth=10, min_samples_leaf=2, n_estimators=50; total time=   2.1s
[CV] END ..max_depth=10, min_samples_leaf=2, n_estimators=50; total time=   2.2s
[CV] END ..max_depth=12, min_samples_leaf=8, n_estimators=50; total time=   0.6s
[CV] END ..max_depth=12, min_samples_leaf=8, n_estimators=50; total time=   0.7s
[CV] END ..max_depth=12, min_samples_leaf=8, n_estimators=50; total time=   0.6s
[CV] END ..max_depth=12, min_samples_leaf=8, n_estimators=50; total time=   0.7s
[CV] END ..max_depth=12, min_samples_leaf=8, n_estimators=50; total time=   0.5s
[CV] END .max_depth=15, min_samples_leaf=8, n_estimators=100; total time=   1.3s
[CV] END .max_depth=15, min_samples_leaf=8, n_e

RandomizedSearchCV(estimator=RandomForestRegressor(random_state=42), n_iter=20,
                   param_distributions={'max_depth': [5, 8, 10, 12, 15],
                                        'min_samples_leaf': [2, 4, 6, 8, 10],
                                        'n_estimators': [10, 50, 100]},
                   verbose=2)

In [75]:
best_model = random_search.best_estimator_
# Train the regressor
best_model.fit(X_train, y_train)
 
# Make predictions on the test set
y_pred = best_model.predict(X_test)
 
# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
 
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 101.4922663714355
R-squared: 0.36005604938983893


In [76]:
y_pred

array([14.11549052, 13.03538274, 37.42336192, 31.73458236,  2.51849097,
        7.20193142, 11.68951895,  5.60134026,  9.07205313,  2.16423332,
        6.46712826, 21.96682576, 14.65575509, 10.87799426,  3.76748222])

In [77]:
y_test

array([18.,  5., 37., 32.,  3.,  0.,  7.,  2.,  7.,  0.,  3.,  3., 16.,
        0., 33.])

## Classication

In [70]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

data_copy = data.copy().dropna()

# list of columns that need to be dropped
to_drop = ['temp_id', 'id', 'calendarDate', 'ageGroup',
            'ausgeschlafen', 'motivation', 'Wach', 'konzentriert',
            'Test_zeit', 'Test_anzahl','prozent_zeit_rang', 'prozent_anzahl'] 

# train with the suitable dataset
features = data_copy.drop(to_drop, axis=1)  # Drop the target variable to isolate the features
target = pd.get_dummies(data_copy['konzentriert'], dtype=int).to_numpy()  # Isolate the target variable

# Convert Timestamp to Unix format
features['sleepStartTimestampLocal'] = features['sleepStartTimestampLocal'].astype('int64') // 10**9
 
features['sleepEndTimestampLocal'] = features['sleepEndTimestampLocal'].astype('int64') // 10**9

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# normalize the features values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
 
# Initialize the Random Forest Regressor
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, criterion='entropy')

parameters = {"n_estimators" : [20, 40, 60, 80],
              "max_depth": [5, 8, 10, 12, 15],
              "min_samples_leaf": [2, 4, 6, 8, 10],
              "n_estimators":[10, 50, 100]}
 
random_search = RandomizedSearchCV(rf_classifier, parameters, n_iter=20, verbose=2)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ..max_depth=10, min_samples_leaf=2, n_estimators=50; total time=   0.2s
[CV] END ..max_depth=10, min_samples_leaf=2, n_estimators=50; total time=   0.2s
[CV] END ..max_depth=10, min_samples_leaf=2, n_estimators=50; total time=   0.2s
[CV] END ..max_depth=10, min_samples_leaf=2, n_estimators=50; total time=   0.2s
[CV] END ..max_depth=10, min_samples_leaf=2, n_estimators=50; total time=   0.2s
[CV] END .max_depth=10, min_samples_leaf=4, n_estimators=100; total time=   0.4s
[CV] END .max_depth=10, min_samples_leaf=4, n_estimators=100; total time=   0.4s
[CV] END .max_depth=10, min_samples_leaf=4, n_estimators=100; total time=   0.5s
[CV] END .max_depth=10, min_samples_leaf=4, n_estimators=100; total time=   0.3s
[CV] END .max_depth=10, min_samples_leaf=4, n_estimators=100; total time=   0.3s
[CV] END ..max_depth=8, min_samples_leaf=10, n_estimators=10; total time=   0.0s
[CV] END ..max_depth=8, min_samples_leaf=10, n_

RandomizedSearchCV(estimator=RandomForestClassifier(criterion='entropy',
                                                    random_state=42),
                   n_iter=20,
                   param_distributions={'max_depth': [5, 8, 10, 12, 15],
                                        'min_samples_leaf': [2, 4, 6, 8, 10],
                                        'n_estimators': [10, 50, 100]},
                   verbose=2)

In [71]:
from sklearn.metrics import accuracy_score #recall_score, classification_report
best_model = random_search.best_estimator_

# Train the classifier
best_model.fit(X_train, y_train)
 
# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print classification report for more detailed performance analysis
#print(classification_report(y_test, y_pred))

Accuracy: 0.00%


In [72]:
y_pred

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [73]:
y_test

array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

### classification without one-hot-encoding

In [64]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

data_copy = data.copy().dropna()

# list of columns that need to be dropped
to_drop = ['temp_id', 'id', 'calendarDate', 'ageGroup',
            'ausgeschlafen', 'motivation', 'Wach', 'konzentriert',
            'Test_zeit', 'Test_anzahl','prozent_zeit_rang', 'prozent_anzahl'] 

# train with the suitable dataset
features = data_copy.drop(to_drop, axis=1)  # Drop the target variable to isolate the features
target = data_copy['konzentriert'].to_numpy() # Isolate the target variable

# Convert Timestamp to Unix format
features['sleepStartTimestampLocal'] = features['sleepStartTimestampLocal'].astype('int64') // 10**9
 
features['sleepEndTimestampLocal'] = features['sleepEndTimestampLocal'].astype('int64') // 10**9

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# normalize the features values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
 
# Initialize the Random Forest Regressor
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, criterion = 'entropy')

parameters = {"n_estimators" : [20, 40, 60, 80],
              "max_depth": [5, 8, 10, 12, 15],
              "min_samples_leaf": [2, 4, 6, 8, 10],
              "n_estimators":[10, 50, 100]}
 
random_search = RandomizedSearchCV(rf_classifier, parameters, n_iter=20, verbose=2)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END max_depth=15, min_samples_leaf=10, n_estimators=100; total time=   0.2s
[CV] END max_depth=15, min_samples_leaf=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=15, min_samples_leaf=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=15, min_samples_leaf=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=15, min_samples_leaf=10, n_estimators=100; total time=   0.1s
[CV] END ..max_depth=15, min_samples_leaf=8, n_estimators=10; total time=   0.0s
[CV] END ..max_depth=15, min_samples_leaf=8, n_estimators=10; total time=   0.0s
[CV] END ..max_depth=15, min_samples_leaf=8, n_estimators=10; total time=   0.0s
[CV] END ..max_depth=15, min_samples_leaf=8, n_estimators=10; total time=   0.0s
[CV] END ..max_depth=15, min_samples_leaf=8, n_estimators=10; total time=   0.0s
[CV] END ..max_depth=5, min_samples_leaf=10, n_estimators=50; total time=   0.0s
[CV] END ..max_depth=5, min_samples_leaf=10, n_

RandomizedSearchCV(estimator=RandomForestClassifier(criterion='entropy',
                                                    random_state=42),
                   n_iter=20,
                   param_distributions={'max_depth': [5, 8, 10, 12, 15],
                                        'min_samples_leaf': [2, 4, 6, 8, 10],
                                        'n_estimators': [10, 50, 100]},
                   verbose=2)

In [65]:
from sklearn.metrics import accuracy_score #recall_score, classification_report
best_model = random_search.best_estimator_

# Train the classifier
best_model.fit(X_train, y_train)
 
# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Calculate accuracy
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse * 100:.2f}%")

Accuracy: 40.00%
MSE: 120.00%


In [58]:
y_test

array([3, 4, 4, 4, 3, 5, 5, 2, 4, 5, 5, 3, 2, 4, 4], dtype=int64)

In [59]:
y_pred

array([3, 3, 3, 3, 5, 3, 3, 3, 4, 5, 3, 3, 4, 3, 3], dtype=int64)

In [45]:
# Make predictions on the test set
y_pred = best_model.predict_proba(X_test)

In [46]:
y_pred

array([[0.10572791, 0.48246428, 0.2335502 , 0.17825761],
       [0.07224677, 0.3514286 , 0.30375525, 0.27256938],
       [0.15071635, 0.49081012, 0.18137878, 0.17709476],
       [0.11587354, 0.54590472, 0.14303349, 0.19518824],
       [0.04290678, 0.32439333, 0.30033729, 0.3323626 ],
       [0.14022909, 0.42521506, 0.23071845, 0.2038374 ],
       [0.11765121, 0.3657097 , 0.3229408 , 0.19369828],
       [0.1128206 , 0.42118815, 0.3651669 , 0.10082435],
       [0.09605275, 0.35285308, 0.37479301, 0.17630116],
       [0.08584706, 0.30973843, 0.26744864, 0.33696587],
       [0.10403466, 0.45282388, 0.2960403 , 0.14710115],
       [0.09941257, 0.45683357, 0.28787027, 0.1558836 ],
       [0.03426357, 0.30907925, 0.47761885, 0.17903833],
       [0.08640464, 0.41835308, 0.2803419 , 0.21490038],
       [0.16844648, 0.35952592, 0.28415913, 0.18786847]])

In [47]:
y_test

5     3
0     4
5     4
1     4
3     3
1     5
11    5
9     2
13    4
5     5
9     5
6     3
10    2
7     4
6     4
Name: konzentriert, dtype: int64